In [2]:
#!/usr/bin/env python
"""
Parallelized Diebold–Li (DNS) yield-curve forecasting — **stable Kalman**
=========================================================================
*Forecasts the last 756 dates (2022-04-13 → 2025-03-05) and saves **one CSV
per horizon** in the requested tidy format:*

```
eval_date,horizon,true_yields,forecast_yields,mae
2022-04-13,5,[...6 vals...],[...6 vals...],0.00123
...
```

No change to `forecast_DNS_KF_explosivcor`.
"""

from __future__ import annotations

import json
import os
import sys
import warnings
from pathlib import Path
from typing import Tuple

import numpy as np
import pandas as pd
from joblib import Parallel, delayed
from sklearn.metrics import mean_absolute_error
from tqdm import tqdm

# ---------------------------------------------------------------------------
# Optional dependency: pykalman
# ---------------------------------------------------------------------------
try:
    from pykalman import KalmanFilter  # type: ignore

    HAVE_PYKALMAN = True
except ModuleNotFoundError:
    HAVE_PYKALMAN = False
    print("[INFO] pykalman missing — forecasts default to random-walk.")

_KALMAN_DISABLED = not HAVE_PYKALMAN
warnings.filterwarnings("once", category=RuntimeWarning)

# ---------------------- DNS helpers ----------------------------------------

def DNS_formula(x: np.ndarray, f: np.ndarray, lam: float) -> np.ndarray:
    l, s, c = f.T
    base = (1 - np.exp(-lam * x)) / (lam * x)
    return l + s * base + c * (base - np.exp(-lam * x))


def DNS_OLS(data: np.ndarray, tau: np.ndarray, lam: float) -> np.ndarray:
    tau = tau.reshape(-1, 1)
    d = lam * tau
    X = np.hstack([
        np.ones_like(tau),
        (1 - np.exp(-d)) / d,
        (1 - np.exp(-d)) / d - np.exp(-d),
    ])
    return (np.linalg.pinv(X.T @ X) @ X.T @ data.T).T


def params_VAR(df: pd.DataFrame) -> Tuple[np.ndarray, np.ndarray]:
    k = df.shape[1]
    if df.shape[0] < 3:
        return np.zeros(k), np.eye(k)
    try:
        from statsmodels.tsa.api import VAR
        fit = VAR(df).fit(1)
        P = fit.params.values
        return P[0], P[1:].T
    except Exception as exc:
        warnings.warn(f"VAR failed ({exc}); using RW.", RuntimeWarning)
        return np.zeros(k), np.eye(k)

# ---------------------- Kalman wrapper -------------------------------------

def kalman_forecast(y_hist: pd.DataFrame, tau: np.ndarray, lam: float,
                    state_init: np.ndarray, offset_init: np.ndarray,
                    trans_init: np.ndarray, steps: int) -> np.ndarray:
    global _KALMAN_DISABLED
    if _KALMAN_DISABLED:
        return np.tile(state_init, (steps, 1))
    try:
        t = tau.reshape(-1, 1)
        dat = y_hist.to_numpy()
        kf = KalmanFilter(
            transition_matrices=trans_init,
            observation_matrices=np.hstack((np.ones((t.size, 1)),
                                             (1 - np.exp(-lam * t)) / (lam * t),
                                             (1 - np.exp(-lam * t)) / (lam * t) - np.exp(-lam * t))),
            transition_offsets=offset_init.reshape(-1),
            observation_offsets=np.zeros(dat.shape[1]),
            transition_covariance=np.eye(3),
            observation_covariance=np.eye(dat.shape[1]),
            initial_state_mean=state_init,
            initial_state_covariance=np.eye(3),
        )
        smooth, _ = kf.smooth(dat)
        A, offs = kf.transition_matrices, kf.transition_offsets
        fc = np.zeros((steps, 3))
        last = smooth[-1]
        for i in range(steps):
            fc[i] = last @ A.T + offs
            last = fc[i]
        return fc
    except Exception as exc:
        warnings.warn(f"Kalman disabled ({exc}); using RW.", RuntimeWarning)
        _KALMAN_DISABLED = True
        return np.tile(state_init, (steps, 1))

# ---------------------- Config ---------------------------------------------
WINDOW = 3 * 252
HORIZONS = [1, 5, 21, 63, 252]
LAM = 0.496
TAU = np.array([0.25, 0.5, 1, 3, 5, 10])
TAU_ROW = TAU.reshape(1, -1)
CORES = min(80, os.cpu_count() or 1)

# ---------------------- Data slice -----------------------------------------
CSV = Path("Y_df.csv")
if not CSV.exists():
    sys.exit(f"File {CSV} not found.")

y_df = pd.read_csv(CSV, index_col=0, parse_dates=True)
IDX, Y, COLS = y_df.index, y_df.values, y_df.columns.tolist()
N = len(IDX)

# Ensure indices are valid for each horizon
ALL_INDICES = {
    h: list(range(WINDOW + h, N)) for h in HORIZONS
}
UNION_INDICES = sorted(set(i for indices in ALL_INDICES.values() for i in indices))

# ---------------------- Helper ---------------------------------------------

def hist_slice(i: int, h: int):
    end = i - h
    start = end - WINDOW
    return Y[start:end] if start >= 0 else Y[0:1]  # fallback if insufficient history

# ---------------------- Worker ---------------------------------------------

def worker(i: int):
    y_t = Y[i]
    date = IDX[i]
    out = []
    for h in HORIZONS:
        if i < WINDOW + h:
            continue
        y_hist = hist_slice(i, h)
        betas = DNS_OLS(y_hist, TAU, LAM)
        offs, A = params_VAR(pd.DataFrame(betas))
        fc_b = kalman_forecast(pd.DataFrame(y_hist), TAU_ROW, LAM, betas[-1], offs, A, h)
        y_p = DNS_formula(TAU, fc_b[-1], LAM)
        out.append({
            "eval_date": date.strftime("%Y-%m-%d"),
            "horizon": h,
            "true_yields": y_t.tolist(),
            "forecast_yields": y_p.tolist(),
            "mae": float(mean_absolute_error(y_t, y_p)),
        })
    return i, out

# ---------------------- Main -----------------------------------------------

def main():
    print(f"Forecasting {len(UNION_INDICES)} dates on {CORES} cores…")
    res = Parallel(n_jobs=CORES, backend="loky")(
        delayed(worker)(i) for i in tqdm(UNION_INDICES, desc="DNS Forecast")
    )
    res.sort(key=lambda x: x[0])

    # Flatten results per horizon
    by_h: dict[int, list[dict]] = {h: [] for h in HORIZONS}
    for _, outs in res:
        for rec in outs:
            by_h[rec["horizon"]].append(rec)

    # Save per-horizon CSVs
    for h, rows in by_h.items():
        pd.DataFrame(rows).to_csv(f"dns_kf_total_h{h}_full_dataset.csv", index=False)

    print("Done ✨ — full dataset forecasts written.")


if __name__ == "__main__":
    main()

Forecasting 4838 dates on 80 cores…



DNS Forecast: 100%|██████████| 4838/4838 [00:54<00:00, 89.51it/s]


Done ✨ — full dataset forecasts written.


In [3]:
pip install pykalman          # to enable Kalman filtering

Note: you may need to restart the kernel to use updated packages.
